# TensorFlow层次结构

In [1]:
import tensorflow as tf
from tensorflow import keras
# from tensorflow.keras import layers
# from tensorflow.keras import losses
# from tensorflow.keras import metrics
# from tensorflow.keras import optimizers

tf.__version__

'2.1.0'

## 低层API示例

**低阶API** 主要包括**张量操作**，**计算图** 和 **自动微分** 等。

In [2]:
#打印时间分割线
@tf.function
def printbar():
    ts = tf.timestamp()
    today_ts = ts % (24 * 60 * 60)

    hour = tf.cast(today_ts // 3600 + 8, tf.int32) % tf.constant(24)
    minite = tf.cast((today_ts % 3600) // 60, tf.int32)
    second = tf.cast(tf.floor(today_ts % 60), tf.int32)

    def timeformat(m):
        if tf.strings.length(tf.strings.format("{}", m))==1:
            return(tf.strings.format("0{}", m))
        else:
            return(tf.strings.format("{}", m))

    timestring = tf.strings.join([timeformat(hour), timeformat(minite), timeformat(second)], separator = ":")
    
    tf.print("=" * 10, end = "")
    tf.print(timestring, end = "")
    tf.print("=" * 10)

In [3]:
s = printbar()

==========08:45:48==========


In [4]:
#样本数量
n = 400

# 生成测试用数据集
X = tf.random.uniform([n, 2], minval=-10, maxval=10) 
print(f"X shape = {X.shape}")
print(f"X head =\n{X[:10]}")
print()

w0 = tf.constant([[2.0], [-1.0]])
print(f"w0 = {w0.shape}, data=\n{w0}")
print()

b0 = tf.constant(3.0)
print(f"b0 = {b0.shape}, data=\n{b0}")
print()

X shape = (400, 2)
X head =
[[ 8.564583    0.20936203]
 [-5.2583647  -8.474501  ]
 [-4.7962046  -2.2101355 ]
 [ 1.8256903   5.8592415 ]
 [-4.295366   -8.950251  ]
 [-4.0162992  -1.1756468 ]
 [-1.6190863   5.7091694 ]
 [ 2.7599125   2.6526718 ]
 [ 5.211215    4.967573  ]
 [ 3.4571533  -7.957039  ]]

w0 = (2, 1), data=
[[ 2.]
 [-1.]]

b0 = (), data=
3.0



In [5]:
normal_penalty = tf.random.normal([n, 1], mean=0.0, stddev=2.0)
normal_penalty[:10]
print(f"normal_penalty shape = {normal_penalty.shape}")
print(f"normal_penalty head =\n{normal_penalty[:10]}")
print()

normal_penalty shape = (400, 1)
normal_penalty head =
[[ 1.3415803 ]
 [ 0.39303797]
 [-0.8475891 ]
 [ 4.3898935 ]
 [-0.25815213]
 [-1.489164  ]
 [-2.5715966 ]
 [ 1.164617  ]
 [ 1.3561958 ]
 [ 2.670498  ]]



In [6]:
Y = X@w0 + b0 + normal_penalty  # @表示矩阵乘法,增加正态扰动
print(f"Y shape = {Y.shape}")
print(f"Y head =\n{Y[:10]}")
print()

Y shape = (400, 1)
Y head =
[[21.261383 ]
 [ 1.3508093]
 [-5.2298627]
 [ 5.1820326]
 [ 3.101367 ]
 [-5.3461156]
 [-8.518938 ]
 [ 7.03177  ]
 [ 9.811052 ]
 [20.541843 ]]



In [7]:
#使用动态图调试

w = tf.Variable(tf.random.normal(w0.shape))
b = tf.Variable(0.0)

def train(epoches):
    for epoch in tf.range(1, epoches + 1):
        with tf.GradientTape() as tape:
            #正向传播求损失
            Y_hat = X@w + b
            # 损失函数(目标函数)
            loss = tf.squeeze(tf.transpose(Y - Y_hat) @ (Y - Y_hat)) / (2.0 * n)   

        # 反向传播求梯度
        dloss_dw, dloss_db = tape.gradient(loss, [w, b])
        # 梯度下降法更新参数
        w.assign(w - 0.0001 * dloss_dw)
        b.assign(b - 0.0001 * dloss_db)
        if epoch % 1000 == 0:
            printbar()
            tf.print("epoch =", epoch, " loss =", loss,)
            tf.print("w =\n", w)
            tf.print("b =\n", b)
            tf.print("")

train(5000)

==========08:45:50==========
epoch = 1000  loss = 5.69667673
w =
 [[1.95621347]
 [-0.864533842]]
b =
 0.278123409

==========08:45:52==========
epoch = 2000  loss = 4.74324751
w =
 [[2.0171876]
 [-0.976967812]]
b =
 0.532167137

==========08:45:54==========
epoch = 3000  loss = 4.21288157
w =
 [[2.01716757]
 [-0.981756806]]
b =
 0.762274

==========08:45:56==========
epoch = 4000  loss = 3.77871418
w =
 [[2.0160284]
 [-0.982959628]]
b =
 0.970532179

==========08:45:58==========
epoch = 5000  loss = 3.42311835
w =
 [[2.01500511]
 [-0.983940899]]
b =
 1.15900707



In [8]:
##使用autograph机制转换成静态图加速

w = tf.Variable(tf.random.normal(w0.shape))
b = tf.Variable(0.0)

@tf.function
def train(epoches):
    for epoch in tf.range(1,epoches+1):
        with tf.GradientTape() as tape:
            #正向传播求损失
            Y_hat = X@w + b
            loss = tf.squeeze(tf.transpose(Y-Y_hat)@(Y-Y_hat))/(2.0*n)   

        # 反向传播求梯度
        dloss_dw,dloss_db = tape.gradient(loss,[w,b])
        # 梯度下降法更新参数
        w.assign(w - 0.001*dloss_dw)
        b.assign(b - 0.001*dloss_db)
        if epoch%1000 == 0:
            printbar()
            tf.print("epoch =",epoch," loss =",loss,)
            tf.print("w =",w)
            tf.print("b =",b)
            tf.print("")
train(5000)

==========08:45:58==========
epoch = 1000  loss = 2.40704346
w = [[2.01104116]
 [-0.987616]]
b = 1.86533523

==========08:45:59==========
epoch = 2000  loss = 1.89424551
w = [[2.00720048]
 [-0.991193414]]
b = 2.55331

==========08:45:59==========
epoch = 3000  loss = 1.82465148
w = [[2.00578475]
 [-0.992511153]]
b = 2.80675554

==========08:45:59==========
epoch = 4000  loss = 1.81520629
w = [[2.00526476]
 [-0.992996752]]
b = 2.90012336

==========08:45:59==========
epoch = 5000  loss = 1.81392479
w = [[2.0050714]
 [-0.993176222]]
b = 2.93451929



## 中阶API示例


**中阶API** 主要包括设置 **各种模型层**、**损失函数**、**优化器**、**数据管道** 和 **特征列** 等等。

In [9]:
#样本数量
n = 800

# 生成测试用数据集
X = tf.random.uniform([n, 2], minval=-10, maxval=10) 

w0 = tf.constant([[2.0], [-1.0]])
b0 = tf.constant(3.0)

Y = X @ w0 + b0 + tf.random.normal([n, 1], mean=0.0, stddev=2.0)  # @表示矩阵乘法,增加正态扰动

print(f"w0 shape = {w0.shape}")
print(f"w0 =\n{w0}")
print()
print(f"b0 shape = {b0.shape}")
print(f"b0 = {b0}")
print()
print(f"X shape = {X.shape}")
print(f"X head =\n{X[:10]}")
print()
print(f"Y shape = {Y.shape}")
print(f"Y head =\n{Y[:10]}")
print()

w0 shape = (2, 1)
w0 =
[[ 2.]
 [-1.]]

b0 shape = ()
b0 = 3.0

X shape = (800, 2)
X head =
[[ 7.5004997   5.7677536 ]
 [ 8.66337    -4.95497   ]
 [-1.4182549  -5.5375457 ]
 [ 2.0937061  -0.09067059]
 [ 3.1333466  -6.932769  ]
 [ 3.7038784  -9.357958  ]
 [ 7.754307    5.937538  ]
 [ 1.9160957   6.206255  ]
 [ 9.265776    0.23511887]
 [ 5.2307463  -0.39458275]]

Y shape = (800, 1)
Y head =
[[12.70994  ]
 [23.17971  ]
 [ 3.7820983]
 [ 9.02637  ]
 [16.671679 ]
 [21.872562 ]
 [11.417866 ]
 [ 1.8504803]
 [19.674597 ]
 [12.413099 ]]



In [10]:
#构建输入数据管道
ds = tf.data.Dataset.from_tensor_slices((X, Y)) \
     .shuffle(buffer_size = 1000).batch(100) \
     .prefetch(tf.data.experimental.AUTOTUNE)  

#定义优化器
optimizer = keras.optimizers.SGD(learning_rate=0.001)

ds

<PrefetchDataset shapes: ((None, 2), (None, 1)), types: (tf.float32, tf.float32)>

In [11]:
linear = keras.layers.Dense(units = 1)
linear.build(input_shape = (2, )) 

@tf.function
def train(epoches):
    for epoch in tf.range(1, epoches + 1):
        L = tf.constant(0.0) #使用L记录loss值
        for X_batch, Y_batch in ds:
            with tf.GradientTape() as tape:
                Y_hat = linear(X_batch)
                loss = keras.losses.mean_squared_error(tf.reshape(Y_hat, [-1]), tf.reshape(Y_batch, [-1]))
            grads = tape.gradient(loss, linear.variables)
            optimizer.apply_gradients(zip(grads, linear.variables))
            L = loss

        if(epoch % 100 == 0):
            printbar()
            tf.print("epoch =", epoch, ", loss =", L)
            tf.print("w =\n", linear.kernel)
            tf.print("b =", linear.bias)
            tf.print("")

train(500)

==========08:46:04==========
epoch = 100 , loss = 5.61641741
w =
 [[2.02299]
 [-0.991715]]
b = [2.44163775]

==========08:46:08==========
epoch = 200 , loss = 4.1973815
w =
 [[2.01992178]
 [-0.989192605]]
b = [2.9430294]

==========08:46:13==========
epoch = 300 , loss = 3.57325673
w =
 [[2.02173185]
 [-0.987432599]]
b = [3.04419]

==========08:46:17==========
epoch = 400 , loss = 3.83709264
w =
 [[2.02009773]
 [-0.988284588]]
b = [3.06463099]

==========08:46:21==========
epoch = 500 , loss = 3.72241354
w =
 [[2.02127457]
 [-0.988627136]]
b = [3.06869936]



## 高阶API示例

**高阶API** 主要为 *tf.keras.models* 提供的模型类接口。

使用Keras接口有以下3种方式构建模型：
1. 使用Sequential按层顺序构建模型；
2. 使用函数式API构建任意结构模型；
3. 继承Model基类构建自定义模型；

### keras.models.Sequential

In [15]:
#样本数量
n = 800

# 生成测试用数据集
X = tf.random.uniform([n, 2], minval=-10, maxval=10) 
w0 = tf.constant([[2.0], [-1.0]])
b0 = tf.constant(3.0)

Y = X@w0 + b0 + tf.random.normal([n, 1], mean=0.0, stddev=2.0)  # @表示矩阵乘法,增加正态扰动

print(f"w0 shape = {w0.shape}")
print(f"w0 =\n{w0}")
print()
print(f"b0 shape = {b0.shape}")
print(f"b0 = {b0}")
print()
print(f"X shape = {X.shape}")
print(f"X head =\n{X[:10]}")
print()
print(f"Y shape = {Y.shape}")
print(f"Y head =\n{Y[:10]}")
print()

w0 shape = (2, 1)
w0 =
[[ 2.]
 [-1.]]

b0 shape = ()
b0 = 3.0

X shape = (800, 2)
X head =
[[ 2.298603   -3.0598927 ]
 [-5.310471   -9.0469    ]
 [-5.8309317  -2.052095  ]
 [-6.204927    3.9892693 ]
 [-7.5203967   7.8968925 ]
 [-0.44991684  0.7949209 ]
 [ 8.628441    3.607974  ]
 [ 3.73672     8.150255  ]
 [-1.7515545  -6.5004706 ]
 [ 7.695341    5.374346  ]]

Y shape = (800, 1)
Y head =
[[ 12.469965 ]
 [  0.1427896]
 [ -6.081075 ]
 [-15.3535595]
 [-22.368181 ]
 [  0.81478  ]
 [ 15.236221 ]
 [  5.1185064]
 [  4.0363536]
 [ 15.379247 ]]



In [16]:
keras.backend.clear_session()

linear = keras.models.Sequential()
linear.add(keras.layers.Dense(1, input_shape =(2, )))
linear.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                (None, 1)                 3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [18]:
### 使用fit方法进行训练

linear.compile(optimizer="adam", loss="mse", metrics=["mae"])
linear.fit(X, Y, batch_size = 20, epochs = 200)  

tf.print("w =\n", linear.layers[0].kernel)
tf.print("b = ", linear.layers[0].bias)

Train on 800 samples
Epoch 1/200
800/800 [==============================] - 0s 356us/sample - loss: 3.5310 - mae: 1.5204
Epoch 2/200
800/800 [==============================] - 0s 72us/sample - loss: 3.5285 - mae: 1.5195
Epoch 3/200
800/800 [==============================] - 0s 74us/sample - loss: 3.5306 - mae: 1.5203
Epoch 4/200
800/800 [==============================] - 0s 79us/sample - loss: 3.5301 - mae: 1.5202
Epoch 5/200
800/800 [==============================] - 0s 82us/sample - loss: 3.5286 - mae: 1.5199
Epoch 6/200
800/800 [==============================] - 0s 81us/sample - loss: 3.5303 - mae: 1.5201
Epoch 7/200
800/800 [==============================] - 0s 75us/sample - loss: 3.5303 - mae: 1.5202
Epoch 8/200
800/800 [==============================] - 0s 88us/sample - loss: 3.5292 - mae: 1.5202
Epoch 9/200
800/800 [==============================] - 0s 76us/sample - loss: 3.5303 - mae: 1.5206
Epoch 10/200
800/800 [==============================] - 0s 72us/sample - loss: 3.5297 -

### 继承 Model基类 构建自定义模型

In [19]:
#样本数量
n = 800

# 生成测试用数据集
X = tf.random.uniform([n, 2], minval=-10, maxval=10) 
w0 = tf.constant([[2.0], [-1.0]])
b0 = tf.constant(3.0)

Y = X@w0 + b0 + tf.random.normal([n, 1], mean=0.0, stddev=2.0)  # @表示矩阵乘法,增加正态扰动

ds_train = tf.data.Dataset.from_tensor_slices((X[0:n*3//4,:],Y[0:n*3//4,:])) \
     .shuffle(buffer_size = 1000).batch(20) \
     .prefetch(tf.data.experimental.AUTOTUNE) \
     .cache()

ds_valid = tf.data.Dataset.from_tensor_slices((X[n*3//4:,:],Y[n*3//4:,:])) \
     .shuffle(buffer_size = 1000).batch(20) \
     .prefetch(tf.data.experimental.AUTOTUNE) \
     .cache()

In [20]:
keras.backend.clear_session()

class MyModel(keras.models.Model):
    def __init__(self):
        super(MyModel, self).__init__()

    def build(self, input_shape):
        self.dense1 = keras.layers.Dense(1)   
        super(MyModel,self).build(input_shape)

    def call(self, x):
        y = self.dense1(x)
        return(y)

model = MyModel()
model.build(input_shape =(None,2))
model.summary()

Model: "my_model"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
dense (Dense)                multiple                  3         
Total params: 3
Trainable params: 3
Non-trainable params: 0
_________________________________________________________________


In [23]:
### 自定义训练循环(专家教程)

optimizer = keras.optimizers.Adam()
loss_func = keras.losses.MeanSquaredError()

train_loss = keras.metrics.Mean(name='train_loss')
train_metric = keras.metrics.MeanAbsoluteError(name='train_mae')

valid_loss = keras.metrics.Mean(name='valid_loss')
valid_metric = keras.metrics.MeanAbsoluteError(name='valid_mae')


@tf.function
def train_step(model, features, labels):
    with tf.GradientTape() as tape:
        predictions = model(features)
        loss = loss_func(labels, predictions)
    gradients = tape.gradient(loss, model.trainable_variables)
    optimizer.apply_gradients(zip(gradients, model.trainable_variables))

    train_loss.update_state(loss)
    train_metric.update_state(labels, predictions)

@tf.function
def valid_step(model, features, labels):
    predictions = model(features)
    batch_loss = loss_func(labels, predictions)
    valid_loss.update_state(batch_loss)
    valid_metric.update_state(labels, predictions)


@tf.function
def train_model(model, ds_train, ds_valid, epochs):
    for epoch in tf.range(1, epochs + 1):
        for features, labels in ds_train:
            train_step(model, features, labels)

        for features, labels in ds_valid:
            valid_step(model, features, labels)

        logs = 'Epoch={}, Loss:{}, MAE:{}, Valid Loss:{}, Valid MAE:{}'

        if  epoch % 100 ==0:
            printbar()
            tf.print(tf.strings.format(logs,
            (epoch, train_loss.result(), train_metric.result(), valid_loss.result(), valid_metric.result())))
            tf.print("w=\n", model.layers[0].kernel)
            tf.print("b=", model.layers[0].bias)
            tf.print("")

        train_loss.reset_states()
        valid_loss.reset_states()
        train_metric.reset_states()
        valid_metric.reset_states()

train_model(model, ds_train, ds_valid, 400)

==========08:58:44==========
Epoch=100, Loss:3.58331847, MAE:1.5213474, Valid Loss:4.64950371, Valid MAE:1.69223499
w=
 [[1.99570811]
 [-0.992921174]]
b= [2.98471689]

==========08:58:55==========
Epoch=200, Loss:3.58328462, MAE:1.52133608, Valid Loss:4.64958668, Valid MAE:1.69224811
w=
 [[1.99570811]
 [-0.992920637]]
b= [2.98473048]

==========08:59:05==========
Epoch=300, Loss:3.58325744, MAE:1.52133095, Valid Loss:4.64962721, Valid MAE:1.69222784
w=
 [[1.99570811]
 [-0.992920578]]
b= [2.98473048]

==========08:59:15==========
Epoch=400, Loss:3.58330536, MAE:1.5213629, Valid Loss:4.64963579, Valid MAE:1.69222236
w=
 [[1.99570811]
 [-0.992920578]]
b= [2.98473048]

